
# Non Contextual Word embedding

## Мотивация

* Машинное обучение требует **числового представления** данных.
* Но **слова — это символы**, не числа.
* Нужен способ преобразовать их в векторы, чтобы:

  * сравнивать слова,
  * находить похожие,
  * использовать в моделях NLP.

---

## One-Hot Encoding / TF-IDF

* Простейший способ кодирования слов:

  * Каждое слово — вектор длиной `|V|` (размер словаря)
  * Только одна позиция равна 1, остальные — 0

Пример:

```python
V = ["cat", "dog", "fish"]
cat = [1, 0, 0]
dog = [0, 1, 0]
fish = [0, 0, 1]
```

---

## Проблемы One-Hot Encoding

* **Разреженность:** вектор огромный, почти весь из нулей
* **Отсутствие семантики:** "cat" и "dog" не ближе друг к другу, чем "cat" и "car"
* **Невозможность обобщения:** новые слова не связаны с уже известными

---

## Идея Word Embedding

* Представим каждое слово как **плотный вектор** небольшой размерности (например, 100–1000)
* Цель: слова с **похожим смыслом → близкие векторы**

Пример (векторы 2D для иллюстрации):

| слово | вектор       | смысл           |
| ----- | ------------ | --------------- |
| king  | (0.8, 0.2)   | власть          |
| queen | (0.79, 0.25) | власть + женск. |
| man   | (0.4, 0.1)   | мужской         |
| woman | (0.42, 0.13) | женский         |

---

## Семантические отношения в embedding-пространстве

Векторы отражают смысловые отношения:

```
king - man + woman ≈ queen
```

→ Векторы можно складывать, вычитать, искать аналоги.

---

## Подходы к построению embedding'ов

1. **Модели на основе контекста (Word2Vec, FastText)**
   — слово -> context
2. **Модели на основе статистики (GloVe)**
   — используют глобальную статистику совместных встреч

---


### Word2Vec


[Word2vec](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) - один из первых методов векторного представления слов,  ставших популярным. 

> "Смысл слова определяется контекстом, в котором оно встречается."

(дистрибутивная гипотеза)

Два основных варианта:

1. **CBOW (Continuous Bag of Words)**

   * Предсказывает текущее слово по контексту
   * Пример: `the white cat ___ on the mat` → `sit`

2. **Skip-gram**

   * Предсказывает контекст по текущему слову
   * Пример: `cat` → (`the`, `white`, `sat`, `on`, `mat`)


![Источник - https://arxiv.org/pdf/1301.3781.pdf ](img/w2v.png)


Вероятность слова в контексте можно интерпретировать как:

$$p(w_o|w_I) = \frac{\exp(v′_{w_O}^T \cdot v_{w_I})}{\sum_{w=1}^{W}\exp(v′_{w_i}^T \cdot v_{w_I})}$$

Для оптимизации используется или negative sampling или hierarchical softmax.

# FastText

[fastText](https://fasttext.cc/) - библиотека от Facebook.
- большое количество предтренированныйх моделей для разных языков
- может сопоставлять вектора для слов вне словаря

Принцип работы - слово делится на N-граммы. Каждой N-грамме сопоставляется вектор, для получения векторного представления всего слова вектора N-грамм суммируются. 

**привет** $\rightarrow$ **&lt;пр**, **при**, **рив**, **иве**, **вет**, **ет&gt;**. 

* Учитывает **части слова**, а не только целое
* Понимает **редкие и новые слова**

* Например, "cats" → похоже на "cat"
* Особенно полезен для **морфологически богатых языков** 

In [ ]:
привет -> [, , ,]

hello -> [, , , ,]

лес -> [, , , ]



### GloVe

[GloVe](https://nlp.stanford.edu/pubs/glove.pdf) - имплементация через минимизацию функционала:

$$J = \sum_{i,j=1}^V f(X_{ij}) (w_i \cdot \tilde{w_j} + b_i + \tilde{b_j} - \log X_{ij})$$

где, $X_{ij}$ - матрица взаимовстречаемости слов (сколько раз слово $i$ имело в контексте слова $j$). 

$f(x)= \begin{cases}
    (\frac{x}{x_{max}})^\alpha, & \text{если }  x<x_{max}.\\
    1, & \text{иначе}.
  \end{cases}
$

* Использует **всю статистику корпуса**, а не только локальные окна
* Быстрее обучается на больших текстах
* Хорошо отражает аналогии и семантические связи

## Ограничения классических embeddings

* Один вектор на слово → не различает **омонимы**
  (например, *bank* — «берег» vs «банк»)
* Не учитывает **контекст**
* Для этого появились **контекстные эмбеддинги** (ELMo, BERT, GPT)

---

| Этап      | Пример                    | Особенности              |
| --------- | ------------------------- | ------------------------ |
| 2013–2016 | Word2Vec, GloVe, FastText | статические              |
| 2018+     | ELMo, BERT, GPT           | контекстные              |
| 2020+     | LLMs                      | универсальные embeddings |



* Анализ тональности (sentiment analysis)
* Рекомендательные системы
* Поиск по смыслу (semantic search)
* Классификация текстов
* Кластеризация документов

## Визуализация embeddings

* Используют **t-SNE** или **UMAP** для понижения размерности
* Можно увидеть кластеры:

  * география (St. Petersburg, Berlin, Rome)
  * глаголы (run, walk, swim)
  * персоналии 

### Пример обучения Word2Vec с помощью [gensim](https://github.com/RaRe-Technologies/gensim)


Каждая строчка в архиве - тройка
> **category** \\t **headline** \\t **text**

Где:
- category - категоря новости
- headline - заголовок
- text - текст новости


#### Обработка данных

In [1]:
import gzip
import re

import gensim

from gensim.models import Word2Vec

from dataclasses import dataclass
from typing import Iterator, List

@dataclass
class Text:
    label: str
    title: str
    text: str

# Чтение файла данных
def read_texts(fn: str="data/news.txt.gz") -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))
                    
# Разбиение текста на слова                 
def tokenize_text(text: str) -> List[str]:
    text = text.lower()
    words = re.findall(r'\b\w+\b', text.lower())
    return words

# Текст без знаков припенания (нужен для gensim)
def normalize_text(text: str) -> str:
    return ' '.join(tokenize_text(text))

#### Обучение word2vec

In [ ]:
## Обучение word2vec
# каждый текст - набор слов через пробел
sentences = [tokenize_text(text.text) for text in read_texts()]

# обучаем w2v
w2v = Word2Vec(sentences)

# сохраняем модель
w2v.wv.save_word2vec_format('w2v_vectors.bin')
# пример
w2v.wv.most_similar("новости")

[('р', 0.9689614176750183),
 ('советский', 0.9140341281890869),
 ('итар', 0.8822527527809143),
 ('экспресс', 0.8762419819831848),
 ('marca', 0.8718820214271545),
 ('известия', 0.8707618117332458),
 ('lifenews', 0.8662394881248474),
 ('рбк', 0.855144202709198),
 ('коммерсантъ', 0.8542253375053406),
 ('интерфакс', 0.8522573113441467)]